In [ ]:
google_colab = False
if google_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from transformers import BertTokenizerFast
from src.config import config, MODEL_CONFIG
import numpy as np
from typing import List
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import torch
import os
from torch import Tensor
from src.utils.time import timing

g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
MAX_ENCODING_LENGTH = 512
DATASET_PATH = config.data.benzinga.cleaned

In [6]:
def embed_input(text, tokenizer):
    # Truncation = True as bert can only take inputs of max 512 tokens.
    # return_tensors = "pt" makes the funciton return PyTorch tensors
    # tokenizer.encode_plus specifically returns a dictionary of values instead of just a list of values
    encoding = tokenizer(
        text, 
        add_special_tokens = True, 
        truncation = True, 
        padding = "max_length", 
        max_length = MAX_ENCODING_LENGTH,
        return_attention_mask = True, 
        return_tensors = "pt"
    )
    # input_ids: mapping the words to tokens
    # attention masks: idicates if index is word or padding
    input_ids = encoding['input_ids']
    attention_masks = encoding['attention_mask']
    return input_ids, attention_masks


@timing
def embed_inputs(texts: list, tokenizer) -> tuple[Tensor, Tensor]:
    input_ids = []
    attention_masks = []
    
    pool_obj = ThreadPoolExecutor(max_workers=os.cpu_count())
    ans = pool_obj.map(partial(embed_input, tokenizer=tokenizer), texts)
    input_ids, attention_masks = list(zip(*ans))

    input_ids: Tensor = torch.cat(input_ids, dim=0)
    attention_masks: Tensor = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

In [7]:
def get_text_and_labels(dat: pd.DataFrame, 
                        text_col: str = None,
                        label_col: str = None) -> tuple[List, List]:
    texts = dat.loc[:, text_col].tolist()
    labels = dat.loc[:, label_col].tolist()
    return texts, labels

In [8]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_CONFIG.transformer_hugface_id)

In [9]:
dataset = pd.read_parquet(DATASET_PATH)

ArrowCapacityError: array cannot contain more than 2147483646 bytes, have 2151638076

In [ ]:
# Dummy column
dataset["text_length"] = dataset["parsed_body"].map(lambda x: len(x))

In [ ]:
texts, labels = get_text_and_labels(dat=dataset, 
                                    text_col="parsed_body", 
                                    label_col="text_length")
input_ids, masks = embed_inputs(texts, tokenizer)

In [ ]:
encoding_matrix = np.ndarray(shape=(input_ids.shape[0], 2*MAX_ENCODING_LENGTH+1), dtype=np.int64)
encoding_matrix[:, 0] = dataset.index
encoding_matrix[:, 1:(MAX_ENCODING_LENGTH+1)] = Tensor.numpy(input_ids)
encoding_matrix[:, (MAX_ENCODING_LENGTH+1):] = Tensor.numpy(masks)

In [ ]:
np.save(file=config.data.bzg.encoding_matrix, arr=encoding_matrix.astype("int64"))

In [ ]:
def get_encoding(encoding_matrix_path: str):
    encoding_matrix = np.load(file=encoding_matrix_path)
    index = encoding_matrix[:, 0]
    input_ids = encoding_matrix[:, 1:(MAX_ENCODING_LENGTH+1)]
    masks = encoding_matrix[:, (MAX_ENCODING_LENGTH+1):]
    return index, input_ids, masks

In [ ]:
idx, ii, msks = get_encoding(config.data.bzg.encoding_matrix)
torch.from_numpy(ii)